<a href="https://www.kaggle.com/code/priyanka500/smart-career-application-capstone-project?scriptVersionId=282750105" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

📌 Title: Smart Career Application Assistant (SCAA)

Team Members: Priyanka, Monika Dhukia, Shrishti Yadav

📘 1. Introduction

The Smart Career Application Assistant (SCAA) is an AI-powered agent designed to help job seekers quickly understand their strengths, improve their resumes, and receive career recommendations. This project uses the Google Agent Development Kit (ADK), Gemini LLM models, and tool integrations to perform resume analysis and provide real-time job market insights.

SCAA automates key stages of the job application process, including skill extraction, job role suggestions, resume improvement tips, salary research, and cover letter generation.

📘 2. Project Objective

Many job seekers struggle to understand:

What skills their resume highlights

Which job roles match their profile

How to improve their resume

What skills are in demand today

Current salary trends

This project solves these problems with one intelligent agent that analyzes resumes and provides actionable insights within seconds.

📘 3. What This Notebook Demonstrates (Capstone Requirements)

This notebook implements five ADK concepts (more than the minimum three required):

✔ 1. LLM-powered agent using Gemini
✔ 2. Built-in ADK Tool: google_search
✔ 3. Custom Tool: Python skill extractor
✔ 4. File processing: Automated PDF resume extraction
✔ 5. Observability: run_debug() call with agent trace

These features combined form a full, real-world career assistant system.

📘 4. System Workflow Overview

Load Resume (PDF) from Kaggle input

Extract text using pdfplumber

Extract skills using custom Python tool

Send resume + skills to ADK agent

Agent performs:

Resume summary

Job role recommendations

Resume improvements

google_search tool call for job market trends

Cover letter generation

This creates a complete AI-driven career analysis pipeline.

📘 5. Tools & Technologies Used

Google ADK (Agent Development Kit)

Gemini 2.5 Flash Lite

google_search Tool

Custom Python Skill Extractor

PDF text extraction using pdfplumber

InMemoryRunner for debugging & agent flow

Kaggle Notebook environment

📘 6. Key Features Implemented
✔ Resume PDF Upload & Extraction

The system loads and processes a PDF file directly from the Kaggle dataset.

✔ Skill Extraction Tool

A custom Python tool extracts relevant technical skills from the resume text.

✔ Intelligent Agent Reasoning

The ADK agent analyzes skills and provides smart recommendations.

✔ Google Search Integration

Agent fetches real-time market insights:

Salary trends

In-demand skills

Job descriptions

✔ Cover Letter Generation

The agent creates a personalized, professional cover letter.

📘 7. Results

The Smart Career Application Assistant produces:

Extracted skills

Three-point resume summary

Recommended job roles

Resume improvement tips

Market skill demand

Salary insights

Auto-generated cover letter

These results significantly reduce the time required for job preparation.

📘 8. Conclusion

The Smart Career Application Assistant (SCAA) demonstrates how a single ADK agent can combine local tools, LLM intelligence, and real-time search to build a practical, high-impact application. This project successfully automates career guidance and highlights the power of intelligent agents for real-world productivity.

📘 9. Future Improvements

Add multi-agent system (Resume Agent + Salary Agent + Job Match Agent)

Introduce memory to personalize recommendations over time

Deploy as a web application

Add PDF section detection (Education, Experience, etc.)

Job–Resume Match Scoring

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume/resumeidea.pdf
/kaggle/input/resumemain/resumeidea.pdf


In [ ]:
!adk web


In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ GOOGLE_API_KEY loaded")
except Exception as e:
    print("❌ Please add GOOGLE_API_KEY in Add-ons → Secrets")
    print("Details:", e)
    raise


✅ GOOGLE_API_KEY loaded


In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported")


✅ ADK components imported


In [12]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

print("✅ Retry configured")


✅ Retry configured


In [13]:
import re

SKILLS_DB = [
    "python", "sql", "javascript", "java", "html", "css",
    "pandas", "numpy", "scikit-learn", "tensorflow", "pytorch",
    "docker", "aws", "gcp", "azure",
    "tableau", "power bi", "matplotlib", "seaborn",
    "react", "node"
]

def extract_skills(text: str):
    text = text.lower()
    found = [s for s in SKILLS_DB if s in text]
    return sorted(set(found))


In [14]:
extract_skills("I know Python, SQL, AWS, Tableau and React.")


['aws', 'python', 'react', 'sql', 'tableau']

In [15]:
career_instruction = """
You are SmartCareer, an AI assistant for job seekers.

Your responsibilities:
1. Read user questions or resumes.
2. Extract important skills and infer likely job roles.
3. Give 2–3 concrete resume improvement tips.
4. Use the google_search tool whenever you need up-to-date information
   (skills in demand, salary ranges, market trends, etc.).
5. Always answer clearly with bullet points and short explanations.

If the user pastes a resume:
- First, summarize the profile.
- Then list skills.
- Then suggest suitable roles.
- Then give improvements.
"""

root_agent = Agent(
    name="smart_career_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Smart Career Application Assistant",
    instruction=career_instruction,
    tools=[google_search],
)

print("✅ Smart Career Agent created")


✅ Smart Career Agent created


In [16]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner ready")


✅ Runner ready


In [18]:
# Helper to call the agent
async def ask_agent(prompt: str):
    print("Running agent...\n")
    response = await runner.run_debug(prompt)
    return response

# Safe print for different ADK response shapes
def print_final_answer(response):
    print("\n=== FINAL ANSWER ===\n")
    last_text = None

    if isinstance(response, list):
        # Look through events from last to first
        for event in reversed(response):
            if hasattr(event, "text") and event.text:
                last_text = event.text
                break
            elif hasattr(event, "content") and event.content:
                last_text = event.content
                break
            elif hasattr(event, "data") and isinstance(event.data, dict) and "text" in event.data:
                last_text = event.data["text"]
                break
    else:
        if hasattr(response, "text"):
            last_text = response.text
        elif hasattr(response, "content"):
            last_text = response.content

    if last_text:
        print(last_text)
    else:
        print("⚠️ No final text message found in response.")


In [19]:
#Demo 1: General Career Question
question = "What skills should I highlight for a junior data scientist role in 2025?"

response = await ask_agent(question)
print_final_answer(response)


Running agent...


 ### Created new session: debug_session_id

User > What skills should I highlight for a junior data scientist role in 2025?
smart_career_agent > To succeed as a junior data scientist in 2025, you should highlight a blend of technical and soft skills.

Here are the key skills to emphasize:

*   **Programming Languages:** Proficiency in Python is paramount, given its extensive libraries (Pandas, NumPy, Scikit-learn) for data manipulation, analysis, and machine learning. R is also valuable, especially for statistical analysis and visualization. SQL is essential for data extraction and management from databases.
*   **Statistical Analysis and Mathematics:** A solid understanding of statistics, probability, calculus, and linear algebra is crucial for building robust models and interpreting data correctly.
*   **Machine Learning:** Knowledge of core machine learning algorithms (e.g., regression, classification, clustering), including their implementation and evaluation, is

In [20]:
#Demo 2: Resume Analyzer
resume_text = """
Priyanka:
Experienced in Python, SQL, pandas, scikit-learn.
Built dashboards using matplotlib and seaborn.
Used Docker to deploy models to cloud.
"""

skills = extract_skills(resume_text)

prompt = f"""
You are SmartCareer.

Here is a resume:

{resume_text}

First:
1. Summarize this candidate in 3 short bullet points.
2. Use this extracted skills list: {skills}

Then:
3. Suggest 2 suitable job roles and explain why.
4. Give 3 concrete improvements to the resume.
5. If helpful, use google_search to check which of these skills are in demand in 2025.
"""

response = await ask_agent(prompt)
print_final_answer(response)


Running agent...


 ### Continue session: debug_session_id

User > 
You are SmartCareer.

Here is a resume:


Priyanka:
Experienced in Python, SQL, pandas, scikit-learn.
Built dashboards using matplotlib and seaborn.
Used Docker to deploy models to cloud.


First:
1. Summarize this candidate in 3 short bullet points.
2. Use this extracted skills list: ['docker', 'matplotlib', 'pandas', 'python', 'scikit-learn', 'seaborn', 'sql']

Then:
3. Suggest 2 suitable job roles and explain why.
4. Give 3 concrete improvements to the resume.
5. If helpful, use google_search to check which of these skills are in demand in 2025.

smart_career_agent > Here's an analysis of Priyanka's resume and suggestions for improvement:

**Candidate Summary:**

*   Experienced in core Python data science libraries including pandas and scikit-learn.
*   Skilled in data visualization with matplotlib and seaborn, and database interaction with SQL.
*   Possesses deployment experience using Docker for cloud-based model

In [21]:
#Demo 3: Salary Checker
prompt = """
Using google_search, find the typical salary range for a junior data scientist
in the USA in 2025. Then summarize it in a few bullet points.
"""

response = await ask_agent(prompt)
print_final_answer(response)


Running agent...


 ### Continue session: debug_session_id

User > 
Using google_search, find the typical salary range for a junior data scientist
in the USA in 2025. Then summarize it in a few bullet points.

smart_career_agent > Here's a summary of the typical salary range for a junior data scientist in the USA in 2025:

*   **Average Base Salary:** The average annual salary for a Junior Data Scientist in the US is reported to be between **$70,000 and $165,000**.
*   **Total Compensation:** This can increase with additional cash compensation, with some sources indicating an average total compensation ranging from **$93,000 to $104,500**.
*   **Salary Range:** While the majority of junior data scientist salaries fall between **$80,000 to $170,000**, the lower end can start as low as **$46,000** and the higher end can reach up to **$243,500** for top earners.
*   **Entry-Level (Less than 1 Year Experience):** For those with less than one year of experience, the average salary is approx

In [22]:
#Demo 4: Cover Letter Generator
name = "Priyanka"
target_role = "Junior Data Scientist"
skills_list = extract_skills(resume_text)

prompt = f"""
Write a short, professional cover letter for {name} applying to a {target_role} role.

Use these skills: {skills_list}

Structure:
- 1 short intro paragraph
- 1 paragraph about skills + impact
- 1 closing paragraph

Keep it under 200 words.
"""

response = await ask_agent(prompt)
print_final_answer(response)


Running agent...


 ### Continue session: debug_session_id

User > 
Write a short, professional cover letter for Priyanka applying to a Junior Data Scientist role.

Use these skills: ['docker', 'matplotlib', 'pandas', 'python', 'scikit-learn', 'seaborn', 'sql']

Structure:
- 1 short intro paragraph
- 1 paragraph about skills + impact
- 1 closing paragraph

Keep it under 200 words.

smart_career_agent > [Hiring Manager Name] (If known, otherwise use title)
[Hiring Manager Title]
[Company Name]
[Company Address]

Dear [Mr./Ms./Mx. Last Name or Hiring Manager],

I am writing to express my enthusiastic interest in the Junior Data Scientist position at [Company Name], as advertised on [Platform where you saw the advertisement]. With a strong foundation in data science principles and practical experience in key technologies, I am eager to contribute to your team.

My technical proficiencies include Python, SQL, and core libraries such as Pandas and Scikit-learn for data manipulation and mode

In [ ]:

#**************************************************************************
#*****************************************************************************
#********************************************************************************
# if somebody wants to uploade resume pdf file


In [23]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.2 MB/s eta 0:00:00a 0:00:01


In [24]:
from ipywidgets import FileUpload
import pdfplumber

upload_widget = FileUpload(accept='.pdf,.txt', multiple=False)
upload_widget


FileUpload(value=(), accept='.pdf,.txt', description='Upload')

In [25]:
import os
print(os.listdir("/kaggle/working"))
print(os.listdir("../input"))


['.virtual_documents']
['resume', 'resumemain']


In [26]:
import pdfplumber

resume_path = "/kaggle/input/resumemain/resumeidea.pdf"

def load_resume_from_pdf(path):
    try:
        with pdfplumber.open(path) as pdf:
            text = ""
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
        return text
    except Exception as e:
        print("Error reading resume:", e)
        return None

resume_text = load_resume_from_pdf(resume_path)

print("=== RESUME TEXT EXTRACTED ===\n")
print(resume_text[:1500])   # show first part only


=== RESUME TEXT EXTRACTED ===

QUARANTINO
ESPOSITO
GRAPHIC DESIGNER
CONTACT PROFILE
786-555-0167 I am a highly proficient graphic designer with over three years of experience
working in various companies. Throughout my career, I have consistently
someone@example.com demonstrated reliability, competence, and a focused approach to my work.
I have successfully led numerous graphic design projects across diverse
www.excellentwebsite.com areas, including marketing, web design, and other related disciplines. My
expertise encompasses a wide range of design tasks, ensuring high-quality
results that meet the unique needs of each project and client.
LANGUAGES
English EDUCATION HISTORY
Spanish
Master of Fine Arts, Graphic Design, Jan 20XX - June 20XX
Graphic Design Institute
SKILLS
Bachelor of Arts, Graphic Design, Jan 20XX - Sep 20XX
Graphic Design
Clover College of the Arts
Photography
WORK EXPERIENCE
Project Management
Video Editing Lead Graphic Designer, AmCell Mobile, Aug 20XX - Dec 20XX
• P

In [27]:
skills = extract_skills(resume_text)
print("Extracted Skills:", skills)


Extracted Skills: []


In [28]:
prompt = f"""
You are SmartCareer.

Analyze this resume:

{resume_text}

Extracted skills: {skills}

TASKS:
1. Give a 3-bullet summary of this candidate.
2. Suggest 2–3 suitable job roles and explain why.
3. Provide 3 resume improvement recommendations.
4. Use google_search to check which of these skills are most in-demand in 2025.
"""

response = await ask_agent(prompt)
print_final_answer(response)


Running agent...


 ### Continue session: debug_session_id

User > 
You are SmartCareer.

Analyze this resume:

QUARANTINO
ESPOSITO
GRAPHIC DESIGNER
CONTACT PROFILE
786-555-0167 I am a highly proficient graphic designer with over three years of experience
working in various companies. Throughout my career, I have consistently
someone@example.com demonstrated reliability, competence, and a focused approach to my work.
I have successfully led numerous graphic design projects across diverse
www.excellentwebsite.com areas, including marketing, web design, and other related disciplines. My
expertise encompasses a wide range of design tasks, ensuring high-quality
results that meet the unique needs of each project and client.
LANGUAGES
English EDUCATION HISTORY
Spanish
Master of Fine Arts, Graphic Design, Jan 20XX - June 20XX
Graphic Design Institute
SKILLS
Bachelor of Arts, Graphic Design, Jan 20XX - Sep 20XX
Graphic Design
Clover College of the Arts
Photography
WORK EXPERIENCE
Project Manag

In [29]:
prompt = f"""
Write a short, professional cover letter based on this resume:

{resume_text}

Use these skills: {skills}
"""

response = await ask_agent(prompt)
print_final_answer(response)


Running agent...


 ### Continue session: debug_session_id

User > 
Write a short, professional cover letter based on this resume:

QUARANTINO
ESPOSITO
GRAPHIC DESIGNER
CONTACT PROFILE
786-555-0167 I am a highly proficient graphic designer with over three years of experience
working in various companies. Throughout my career, I have consistently
someone@example.com demonstrated reliability, competence, and a focused approach to my work.
I have successfully led numerous graphic design projects across diverse
www.excellentwebsite.com areas, including marketing, web design, and other related disciplines. My
expertise encompasses a wide range of design tasks, ensuring high-quality
results that meet the unique needs of each project and client.
LANGUAGES
English EDUCATION HISTORY
Spanish
Master of Fine Arts, Graphic Design, Jan 20XX - June 20XX
Graphic Design Institute
SKILLS
Bachelor of Arts, Graphic Design, Jan 20XX - Sep 20XX
Graphic Design
Clover College of the Arts
Photography
WORK EXPE